In [1]:
import glob
import os
import zipfile
import pandas as pd

df = pd.DataFrame(columns=['label', 'zip_filename', 'filename_r', 'filename_g', 'filename_b'])
# zipデータセット形式(zipファイル内にチャネルに別れたpngファイルがある。32x32) 10ファイルごとに1zipファイルを作成する
labels = [os.path.basename(x) for x in glob.glob("./data-org/*")]
labels

label_dict = {}
for i in range(len(labels)):
    label_dict[labels[i]] = i
label_dict

{'marker_black': 0,
 'ruler_black': 1,
 'pen_red': 2,
 'pencil_core': 3,
 'pen_green': 4,
 'paste': 5,
 'pen_multiclolor': 6,
 'marker_red': 7}

In [2]:
from PIL import Image
import numpy as np

i = 0
dst_dir = './data-zip'
os.makedirs(dst_dir, exist_ok=True)
files = []
files_r = []
files_g = []
files_b = []

for file_path in glob.glob("./data-org/**/*.JPG", recursive=True):
    zip_filename = f'{i}.zip'
    files.append(zip_filename)
    label = os.path.split(os.path.split(file_path)[0])[1]
    file_name = os.path.basename(file_path)
    base = os.path.splitext(file_name)[0]

    im = np.array(Image.open(file_path))
    im_R = im.copy()
    im_R[:, :, 1] = im_R[:, :, 0]
    im_R[:, :, 2] = im_R[:, :, 0]
    dst_name_r = f'{base}-R.png'
    files_r.append(dst_name_r)
    Image.fromarray(im_R).resize((32,32)).convert('L').save(os.path.join(dst_dir, dst_name_r))

    im_G = im.copy()
    im_G[:, :, 0] = im_G[:, :, 1]
    im_G[:, :, 2] = im_G[:, :, 1]
    dst_name_g = f'{base}-G.png'
    files_g.append(dst_name_g)
    Image.fromarray(im_G).resize((32,32)).convert('L').save(os.path.join(dst_dir, dst_name_g))

    im_B = im.copy()
    im_B[:, :, 0] = im_B[:, :, 2]
    im_B[:, :, 1] = im_B[:, :, 2]
    dst_name_b = f'{base}-B.png'
    files_b.append(dst_name_b)
    Image.fromarray(im_B).resize((32,32)).convert('L').save(os.path.join(dst_dir, dst_name_b))
    df = df.append({'label': label_dict[label], 'zip_filename': os.path.join(dst_dir, zip_filename), 
                    'filename_r': dst_name_r, 'filename_g': dst_name_g, 'filename_b': dst_name_b}, ignore_index=True)
    if len(files) > 100:
        with zipfile.ZipFile(os.path.join(dst_dir, zip_filename), 'w', zipfile.ZIP_DEFLATED) as zip_file:
            for j in range(len(files)):
                zip_file.write(os.path.join(dst_dir, files_r[j]), files_r[j])
                zip_file.write(os.path.join(dst_dir, files_g[j]), files_g[j])
                zip_file.write(os.path.join(dst_dir, files_b[j]), files_b[j])
        files = []
        files_r = []
        files_g = []
        files_b = []
        i += 1

with zipfile.ZipFile(os.path.join(dst_dir, zip_filename), 'w', zipfile.ZIP_DEFLATED) as zip_file:
    for j in range(len(files)):
        zip_file.write(os.path.join(dst_dir, files_r[j]), files_r[j])
        zip_file.write(os.path.join(dst_dir, files_g[j]), files_g[j])
        zip_file.write(os.path.join(dst_dir, files_b[j]), files_b[j])

df.to_csv(os.path.join(dst_dir,'dataset-zip.csv'))
df

,label,zip_filename,filename_r,filename_g,filename_b
0,0,./data-zip/0.zip,IMG_5622-R.png,IMG_5622-G.png,IMG_5622-B.png
1,0,./data-zip/0.zip,IMG_5583-R.png,IMG_5583-G.png,IMG_5583-B.png
2,0,./data-zip/0.zip,IMG_5781-R.png,IMG_5781-G.png,IMG_5781-B.png
3,0,./data-zip/0.zip,IMG_5742-R.png,IMG_5742-G.png,IMG_5742-B.png
4,0,./data-zip/0.zip,IMG_5743-R.png,IMG_5743-G.png,IMG_5743-B.png
...,...,...,...,...,...
600,7,./data-zip/5.zip,IMG_6643-R.png,IMG_6643-G.png,IMG_6643-B.png
601,7,./data-zip/5.zip,IMG_6521-R.png,IMG_6521-G.png,IMG_6521-B.png
602,7,./data-zip/5.zip,IMG_6642-R.png,IMG_6642-G.png,IMG_6642-B.png
603,7,./data-zip/5.zip,IMG_6681-R.png,IMG_6681-G.png,IMG_6681-B.png
